<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/KMG/Titanic2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv')

In [21]:
columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
train = train[columns+['Survived']]
test = test[columns]

mean_age = train['Age'].mean()
mean_fare = train['Fare'].mean()

train['Age'] = train['Age'].fillna(mean_age)
test['Age'] = test['Age'].fillna(mean_age)

train['Fare'] = train['Fare'].fillna(mean_fare)
test['Fare'] = train ['Fare'].fillna(mean_fare)

train = train[train['Parch'] <= 5]
train = train[train['Fare'] <= 300]

train['Sex'] = train['Sex'].apply(lambda x : 0 if x == 'female' else 1)
test['Sex'] = train['Sex'].apply(lambda x : 0 if x == 'female' else 1)

train = pd.get_dummies(train, columns = ['Embarked'], drop_first = True)
test = pd.get_dummies(test, columns = ['Embarked'], drop_first = True)

train_x = train.drop(columns = ['Survived'])
train_y = train['Survived']

from sklearn.model_selection import train_test_split

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.2, random_state=0)

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state = 0)

X_resampled, y_resampled = smote.fit_resample(train_x, list(train_y))

X_resampled['Survived'] = y_resampled
train_dataset = X_resampled

<ipython-input-21-ed0969593e0c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Age'] = train['Age'].fillna(mean_age)
<ipython-input-21-ed0969593e0c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Age'] = test['Age'].fillna(mean_age)
<ipython-input-21-ed0969593e0c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [22]:
# 로지스틱 회귀 모델
import statsmodels.api as sm

from sklearn.metrics import classification_report

from sklearn.metrics import confusion_matrix

formula = """
Survived ~ C(Pclass) + C(Sex) + scale(Age) + scale(SibSp) + scale(Parch) + scale(Fare) + C(Embarked_Q) + C(Embarked_S)
"""

model = sm.Logit.from_formula(formula, data=train_dataset)
result = model.fit()
y_pred = result.predict(val_x)
y_pred = y_pred.apply(lambda x : 1 if x>=0.5 else 0)

print(classification_report(val_y, y_pred))

Optimization terminated successfully.
         Current function value: 0.377463
         Iterations 7
              precision    recall  f1-score   support

           0       0.74      0.79      0.76       103
           1       0.68      0.61      0.64        75

    accuracy                           0.71       178
   macro avg       0.71      0.70      0.70       178
weighted avg       0.71      0.71      0.71       178



In [26]:
# 의사 결정 트리 모델
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=6, random_state=0)
model.fit(train_dataset.drop(columns = 'Survived'), train_dataset['Survived'])
y_pred = model.predict(val_x)
print(confusion_matrix(val_y, y_pred))

print(classification_report(val_y, y_pred))

[[89 14]
 [29 46]]
              precision    recall  f1-score   support

           0       0.75      0.86      0.81       103
           1       0.77      0.61      0.68        75

    accuracy                           0.76       178
   macro avg       0.76      0.74      0.74       178
weighted avg       0.76      0.76      0.75       178



In [27]:
# 랜덤 포레스트 모델
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 200, max_depth = 5, random_state = 0)
model.fit(train.drop(columns = 'Survived'), train['Survived'])

y_pred = model.predict(val_x)
print(confusion_matrix(val_y, y_pred))
print(classification_report(val_y, y_pred))

[[96  7]
 [33 42]]
              precision    recall  f1-score   support

           0       0.74      0.93      0.83       103
           1       0.86      0.56      0.68        75

    accuracy                           0.78       178
   macro avg       0.80      0.75      0.75       178
weighted avg       0.79      0.78      0.76       178



In [28]:
# XGBoost 모델
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators = 200, learning_rate = 0.01, max_depth = 5, random_state = 0)

model.fit(train.drop(columns = 'Survived'), train['Survived'])

y_pred = model.predict(val_x)
print(confusion_matrix(val_y, y_pred))
print(classification_report(val_y, y_pred))

[[94  9]
 [26 49]]
              precision    recall  f1-score   support

           0       0.78      0.91      0.84       103
           1       0.84      0.65      0.74        75

    accuracy                           0.80       178
   macro avg       0.81      0.78      0.79       178
weighted avg       0.81      0.80      0.80       178



In [30]:
# 모델 선택
my_model = XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=5, random_state = 0)
my_model.fit(train_dataset.drop(columns = 'Survived'), train_dataset['Survived'])

XGB_pred = my_model.predict(test)